In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import random as rand
import numpy as np
import scipy
import prior_calc
import posterior_calc
import collections
from networkx.algorithms.traversal.depth_first_search import dfs_tree
import tree_generation
import data_reader
import generate_test_data
import tree_comparison

In [9]:
reload(prior_calc)
reload(generate_test_data)

<module 'generate_test_data' from 'generate_test_data.pyc'>

In [3]:
# METADATA
NUM_TRIALS = 1000
for ALPHA in [0.3, 0.5, 0.7, 0.9]:
    for NUM_CELLS in [58]:
        # This cell evaluates all the priors with error probabilities
        priors = prior_calc.prior_calculator(NUM_CELLS, ALPHA, Btree=100, Bmut=10000, withError=True)
        priors.simulate()
        priors.compute_final_probs()
        prior_probabilities = priors.return_priors()
        
        # Now run the experiment
        for NUM_MUTS in [18]:
            total_score = 0
            for trial in range(1000):
                total_score += perform_experiment(NUM_CELLS, NUM_MUTS, ALPHA, prior_probabilities, error=True)
            score = total_score/float(1000)
            print 'For ALPHA=' + str(ALPHA) + ', NUM_CELLS=' + str(NUM_CELLS) + ', NUM_MUTS= ' + str(NUM_MUTS) + ', score=' + str(score)
                

For ALPHA=0.3, NUM_CELLS=58, NUM_MUTS= 18, score=0.52985620915
For ALPHA=0.5, NUM_CELLS=58, NUM_MUTS= 18, score=0.387215686275
For ALPHA=0.7, NUM_CELLS=58, NUM_MUTS= 18, score=0.282797385621
For ALPHA=0.9, NUM_CELLS=58, NUM_MUTS= 18, score=0.203483660131


In [ ]:
# This cell evaluates all the priors with error probabilities
reload(prior_calc)
priors = prior_calc.prior_calculator(NUM_CELLS, ALPHA, Btree=100, Bmut=10000, withError=True)
priors.simulate()
priors.compute_final_probs()
prior_probabilities = priors.return_priors()

In [45]:
# This cell evaluates all the priors without error probabilities
reload(prior_calc)
priors_pure = prior_calc.prior_calculator(NUM_CELLS, ALPHA, Btree=100, Bmut=10000)
priors_pure.simulate()
priors_pure.compute_final_probs()
prior_probabilities_pure = priors_pure.return_priors()

3.09944152832e-06
5.76702213287
11.4489450455
17.0052092075
22.5690841675
28.2525510788
33.8485150337
39.6583621502
45.5231370926
51.0333631039


In [2]:
reload(generate_test_data)
# This function assumes that the priors have been already calculated
# for the given number of cells and the alpha value. This function performs
# exactly 1 experiment in the following steps:
# 1) Generate a dataset, with a "true" tree (along with ambiguity information)
# 2) Construct the posteriors and our inferred tree
# 3) Calculate how similar those two trees were, taking ambiguities into account
def perform_experiment(num_cells, num_muts, alpha_value, prior_probs, error=False):
    # Generate "fake" data to test
    test_generator = generate_test_data.data_generator(num_cells, num_muts, alpha=alpha_value, withError=error)
    test_generator.initialize_Tk()
    test_generator.create_lineage_tree()
    test_generator.apply_mutations()
    test_data = test_generator.return_genotype_data()
    true_tree = test_generator.construct_true_mut_tree() # true tree to be used for comparison
    true_ambiguities = test_generator.get_ambiguities() # inherent ambiguities in true tree
    
    # Calculate posteriors and construct the tree
    posteriors = posterior_calc.posterior_calculator(test_data, prior_probs, num_cells)
    posteriors.calculate_likelihood()
    posteriors.calculate_posteriors()
    posterior_probabilities = posteriors.return_posteriors()
    tree_generator = tree_generation.tree_generation(posterior_probabilities)
    inferred_tree = tree_generator.get_min_tree()
    
    # Compare the true_tree and the inferred_tree and return the similarity score
    return tree_comparison.compare_mutation_order(true_tree, inferred_tree, true_ambiguities)
    
    

In [ ]:
# This is for using Kim/Simon's true data
mutation_data = data_reader.data_reader('data_mmc2.xlsx')
target = ['PDE4DIP(A->G)', 'NTRK1(A->G)', 'SESN2(C->T)', 'ARHGAP5(G->A)', 'DNAJC17(C->G)', 'USP32(C->T)', 
          'ANAPC1(G->A)', 'RETSAT(C->T)', 'ST13(G->A)', 'DLEC1(T->C)', 'FRG1(G->A)', 'DMXL1(G->A)', 'FAM115C(T->C)', 
          'MLL3(C->T)', 'ABCB5(G->T)', 'ASNS(T->A)', 'PABPC1(C->T)', 'TOP1MT(A->G)']

important_gene = collections.OrderedDict()
for gene in target:
    important_gene[gene] = mutation_data.get_gene_mutations(gene)
data = important_gene